In [1]:
import sys
import os

path = os.path.abspath('../..')
if path not in sys.path:
    sys.path.insert(0, path)
    
sys.path

['/home/justin/Github/aqbuildtools',
 '/home/justin/Github/aqbuildtools/examples/2021RedesignCampaign',
 '/home/justin/anaconda3/envs/aqbt/lib/python38.zip',
 '/home/justin/anaconda3/envs/aqbt/lib/python3.8',
 '/home/justin/anaconda3/envs/aqbt/lib/python3.8/lib-dynload',
 '',
 '/home/justin/anaconda3/envs/aqbt/lib/python3.8/site-packages',
 '/home/justin/anaconda3/envs/aqbt/lib/python3.8/site-packages/IPython/extensions',
 '/home/justin/.ipython']

In [2]:
from aqbt import AquariumBuildTools
aqtools = AquariumBuildTools.from_toml('creds.secret.toml')
aqtools.sessions

aq = aqtools.sessions['production']['aquarium']
aq.set_timeout(70)

In [5]:


registry = aqtools.sessions['production']['registry']


In [8]:
from aqbt.aquarium.resolve_sequences import Resolver

resolver = Resolver(aq, registry)

In [11]:
sample = aq.Sample.find_by_name("TRP-pp2_5'hom__v2")


In [12]:
resolver.resolve_sequence(sample)

Resolving sequence for 'TRP-pp2_5'hom__v2'


<DNASequence (DNASequence) at 140036229335456>

In [15]:
resolver.registry.find_in_cache(sample)

<DNASequence (DNASequence) at 140036229335456>

In [43]:
from aqbt.aquarium.parsers.pgrr_plasmid import parse_name

name = 'pMOD-HO-Kan-pGRR-W8W36-RGR-W20'
seq = parse_name(name)
seq.name = name
seq.annotations['name'] = name

from aqbt.bioadapter import convert


folder = benchling.Folder.find_by_name('2021Campaign')
dna = convert(seq, to='DNASequence', benchling_session=aqtools.sessions['production']['benchling'], benchling_folder_id=folder.id)
dna.save()

<DNASequence (DNASequence) at 140036188536896>

In [41]:
seq.name

'<unknown name>'

In [31]:
benchling = aqtools.sessions['production']['benchling']

In [32]:
folder = benchling.Folder.find_by_name('2021Campaign')


<Folder (Folder) at 140036188995936>